# Multi agent
So far we saw one Agent. In this we will explore multi agent workflow. 

We have 3 agents which are specialized in their task

- Clinvar Agent
- dbsnfp Agent
- Gnomad Agent

We will have one main agent which will interact with user and provide update on these database releases.



#### Load library

In [1]:
from dotenv import load_dotenv
from agents import Runner, trace, function_tool, Agent, OpenAIChatCompletionsModel, SQLiteSession
from openai import AsyncOpenAI

# Needed for gnomad, dbsnfp release notes fetching
import os, requests, time, re
from bs4 import BeautifulSoup
from datetime import datetime
from typing import List, Dict

# Needed for clinvar release notes pdf extraction
import pytesseract
from pdf2image import convert_from_path

#### Load openai key from environment variables

In [2]:
load_dotenv(override=True)

True

#### Define database urls

In [3]:
CLINVAR_RELEASE_NOTES_URL = "https://ftp.ncbi.nlm.nih.gov/pub/clinvar/release_notes/"
GNOMAD_URL = "https://registry.opendata.aws/broad-gnomad/"
DBNSFP_URL = "https://www.dbnsfp.org/releases"

#### Define tools for dbNSFP

A simple python function to fetch DBNSFP release notes using requests and BeautifulSoup


In [4]:

@function_tool
def fetch_dbnsfp_releases():
    """ Fetch the DBNSFP releases page content. """
    url = DBNSFP_URL
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    # extract all texts from html page
    texts = soup.get_text()
    return texts

#### Define tools for Gnomad

A simple python function to fetch gnomad release notes using requests and beautifulsoup

In [ ]:
@function_tool
def fetch_gnomad_releases():
    """ Fetch the Gnomad releases page content. """
    url = GNOMAD_URL
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    # extract all texts from html page
    texts = soup.get_text()
    return texts

#### Define tools for Clinvar

A total of 4 functions for ClinVar release notes fetching and processing
- list_clinvar_release_notes: fetch and list all ClinVar release notes PDFs from the NCBI FTP site
- sort_clinvar_release_notes_url_by_date: find the latest ClinVar release note PDF based on the date in the filename
- download_clinvar_release_note_by_url: download the ClinVar release note PDF from the given URL
- extract_text_from_pdf_ocr: extract text from a PDF using OCR


In [6]:
@function_tool
def list_clinvar_release_notes():
    """Fetch and list ClinVar release notes PDFs from the CLINVAR_RELEASE_NOTES_URL."""
    response = requests.get(CLINVAR_RELEASE_NOTES_URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    pdfs = []
    for link in soup.find_all("a"):
        href = link.get("href")
        if href and href.endswith(".pdf"):
            pdfs.append({
                "filename": href,
                "url": CLINVAR_RELEASE_NOTES_URL + href
            })
    return pdfs

from typing import List, Dict, TypedDict

class ReleaseNote(TypedDict):
    filename: str
    url: str

@function_tool
def sort_clinvar_release_notes_url_by_date(pdfs: List[ReleaseNote]) -> ReleaseNote:
    """Given a list of ClinVar release note PDF urls, sort them based on the date in the filename and return the latest one."""
    latest = None
    latest_date = None
    date_re = re.compile(r'^(\d{8})')
    for pdf in pdfs:
        m = date_re.match(pdf['filename'])
        if m:
            date_str = m.group(1)
            try:
                date = datetime.strptime(date_str, '%Y%m%d')
                if latest_date is None or date > latest_date:
                    latest = pdf
                    latest_date = date
            except Exception:
                continue
    return latest

@function_tool
def download_clinvar_release_note_by_url(url: str, output_dir: str = ".") -> str:
    """ Download the ClinVar release note PDF from the given URL. 
    Download always in the current directory."""
    filename = url.split("/")[-1]
    local_path = os.path.join(output_dir, filename)
    try:
        r = requests.get(url, stream=True)
        r.raise_for_status()
        with open(local_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        return local_path
    except Exception as e:
        print(f"Failed to download {filename}: {e}")
        return None
    
@function_tool
def extract_text_from_pdf_ocr(pdf_path: str) -> str:
    """ Extract text from a PDF using OCR. """
    try:
        images = convert_from_path(pdf_path)
        text = ""
        for img in images:
            text += pytesseract.image_to_string(img)
        time.sleep(1)
        return text
    except Exception as e:
        return f"Failed to extract text with OCR: {e}"

#### Define LLM model

For demo purpose, we will use a local model gpt-oss which has reasoning and tool calling capabilities.

In [7]:
#model = "gpt-4.1-mini"

# Alternatively, you can use a local model
client = AsyncOpenAI(base_url="http://localhost:11434/v1")
model = OpenAIChatCompletionsModel(model = "gpt-oss",openai_client= client)

#### Create dbNSFP Agent

Lets first create dbSNFP agent

In [8]:
instructions_dbnsfp = (
        "You are a specialized agent for dbNSFP. Your task is to use fetch_dbnsfp_releases tool to fetch the latest release notes from dbNSFP database and produce a concise, well-formatted markdown summary containing the latest release version, release date if any, and a bulleted list of major changes or updates"
    )

dbnsfp_agent = Agent(
        name="dbnsfp_agent",
        instructions=instructions_dbnsfp,
        model=model,
        tools=[fetch_dbnsfp_releases]
    )


#### Create Gnomad Agent

Lets create gnomad agent

In [9]:
instructions_gnomad = (
        "You are a specialized agent for Gnomad. Your task is to use fetch_gnomad_releases tool to fetch the latest release notes from Gnomad database and produce a concise, well-formatted markdown summary containing the latest release version, release date if any, and a bulleted list of major changes or updates"
    )

gnomad_agent = Agent(
        name="gnomad_agent",
        instructions=instructions_gnomad,
        model=model,
        tools=[fetch_gnomad_releases]
    )

#### Create Clinvar Agent

Lets create Clinvar agent

In [ ]:
# Provide detailed instructions for the agent. It is better to specify what all tools are available to the agent. You can even guide the agent on how to use the tools. 

instructions_clinvar = ("You are a specialized agent for ClinVar release notes analysis. You have access to the following tools: list_clinvar_release_notes, sort_clinvar_release_notes_url_by_date, download_clinvar_release_note_by_url, and extract_text_from_pdf_ocr.  Use these tools to: list all available release notes, identify the latest one, download it, extract its text using OCR, and always summarize the extracted content in markdown format. Also check if any mention of xml format change or tag updates present in release notes. If present, explicitely mention those tag updates in the markdown summary. If not, state that there are no xml tag updates.")


clinvar_agent = Agent(name="Clinvar_Release_Notes_Agent",
                    instructions=instructions_clinvar,
                    model=model,
                    tools=[list_clinvar_release_notes, sort_clinvar_release_notes_url_by_date, download_clinvar_release_note_by_url, extract_text_from_pdf_ocr])


#### Convert these Agents into Tools so that main Agent can use these sub-agents

Just use as_tool function and give name and description

In [11]:
dbnsfp_tool = dbnsfp_agent.as_tool(
    tool_name="dbnsfp_release_notes_summarizer",
    tool_description="Fetch and summarize the latest dbNSFP release notes in markdown format.")

gnomad_tool = gnomad_agent.as_tool(
    tool_name="gnomad_release_notes_summarizer",
    tool_description="Fetch and summarize the latest Gnomad release notes in markdown format.")

clinvar_tool = clinvar_agent.as_tool(
    tool_name="clinvar_release_notes_summarizer",  
    tool_description="Fetch and summarize the latest ClinVar release notes in markdown format.")

#### Define Main agent

In [12]:
summarizer_agent = Agent(
    name="Release_Notes_Summarizer_Agent",
    instructions="You are a helpful assistant who is expert in summarizing release notes from various bioinformatics databases such as dbNSFP, Gnomad, and ClinVar into concise markdown format. Use the provided tools to fetch and summarize the release notes as per user requests.",
    model=model, 
    tools=[dbnsfp_tool, gnomad_tool, clinvar_tool]
)

#### Run main agent in chat mode with session enabled

In [ ]:
session = SQLiteSession("release_notes_summarizer_session")

async def chat(message, history):
    result = await Runner.run(summarizer_agent, message, session=session)
    return result.final_output


import gradio as gr
gr.ChatInterface(
    chat,
    title="Release Notes Summarizer",
    description="Chat with the agent to get release notes summaries from various bioinformatics databases such as dbNSFP, Gnomad, and ClinVar."
).launch()


/mnt/c/Users/PriyabrataPanigrahi/Downloads/ai/prpanigrahi/agent-in-bioinfo/course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[non-fatal] Tracing: server error 503, retrying.
